In [275]:
import pandas as pd
pd.set_option("display.max_rows", 500)
import math
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import warnings
warnings.filterwarnings(action="ignore")

In [283]:
# Read df_crawled_routes
df_crawled_routes = pd.read_excel("flight_data.xlsx").drop_duplicates()

# Read df_routes
df_routes = pd.read_excel("flight_routes.xlsx")
df_routes.columns = df_routes.columns.str.lower().str.replace(" ", "_")

# Read the airport data
df_airport_data = pd.read_excel("airport_data.xlsx")
# Add the latitude and longitude to the dataset
df_airport_data[["latitude", "longitude"]] = df_airport_data["Location"].str.split(",", 1, expand=True)

In [284]:
# Create a function to calculate the distance between two airports based on their latitude and longitude
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on the Earth's surface
    given their latitude and longitude coordinates.
    """
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    radius = 6371  # Radius of the Earth in kilometers
    distance = radius * c

    return round(distance, 2)

In [285]:
# Add the latitude and longitude fields of the *origin* airport
df_crawled_routes = pd.merge(
    left=df_crawled_routes,
    right=df_airport_data[["IataCode", "latitude", "longitude"]].drop_duplicates("IataCode"),
    left_on="origin_city_id",
    right_on="IataCode",
    how="left"
).drop(["IataCode"], axis=1)

# Add the latitude and longitude fields of the *destination* airport
df_crawled_routes = pd.merge(
    left=df_crawled_routes,
    right=df_airport_data[["IataCode", "latitude", "longitude"]].drop_duplicates("IataCode"),
    left_on="arrival_city_id",
    right_on="IataCode",
    how="left",
    suffixes=["_origin_city", "_arrival_city"]
).drop(["IataCode"], axis=1)

# Change the lat/long columns to type float
df_crawled_routes[["latitude_origin_city", "longitude_origin_city", "latitude_arrival_city", "longitude_arrival_city"]] = \
    df_crawled_routes[["latitude_origin_city", "longitude_origin_city", "latitude_arrival_city", "longitude_arrival_city"]].apply(lambda x: pd.to_numeric(x))

# Add the route competition status from df_routes
df_crawled_routes = pd.merge(
    left=df_crawled_routes,
    right=df_routes,
    left_on=["origin_city_search_term", "arrival_city_search_term"],
    right_on=["departure_city", "arrival_city"],
    how="left"
)

# Change all instances of Wizz Air {x} to just Wizz Air and replace any occurrence of a dot or space character in the "competitor" with an underscore. Finally, change all competitor names to lower case
df_crawled_routes["competitor"] = df_crawled_routes["competitor"].apply(lambda x: "Wizz" if "Wizz" in x else x).apply(lambda x: x.replace(".", "_").replace(" ", "_").lower())

In [286]:
# Add the distance field
df_crawled_routes["flight_distance_km"] = df_crawled_routes.\
    apply(lambda x: 
        haversine_distance(
            lat1=x["latitude_origin_city"],
            lon1=x["longitude_origin_city"],
            lat2=x["latitude_arrival_city"],
            lon2=x["longitude_arrival_city"]
        ), axis=1
    )
df_crawled_routes

,price_eur,origin_airport_name,origin_airport_display_code,arrival_airport_name,arrival_airport_display_code,flight_departure_time,flight_arrival_time,competitor,flight_duration,origin_city_search_term,...,arrival_city_id,crawling_date,latitude_origin_city,longitude_origin_city,latitude_arrival_city,longitude_arrival_city,departure_city,arrival_city,route_competition_status,flight_distance_km
0,213.10,Aalborg,AAL,Kaunas,KUN,2023-06-04T13:05:00,2023-06-04T17:55:00,dat,230,Aalborg,...,KUN,2023-06-04,57.093056,9.850000,54.963919,24.084778,Aalborg,Kaunas,Monopoly always,913.71
1,241.00,Aalborg,AAL,Kaunas,KUN,2023-06-04T13:20:00,2023-06-04T17:55:00,norwegian,215,Aalborg,...,KUN,2023-06-04,57.093056,9.850000,54.963919,24.084778,Aalborg,Kaunas,Monopoly always,913.71
2,228.32,Aalborg,AAL,Kaunas,KUN,2023-06-04T12:55:00,2023-06-04T17:55:00,scandinavian_airlines,240,Aalborg,...,KUN,2023-06-04,57.093056,9.850000,54.963919,24.084778,Aalborg,Kaunas,Monopoly always,913.71
3,152.83,Aalborg,AAL,Kaunas,KUN,2023-06-04T09:05:00,2023-06-04T17:55:00,norwegian,470,Aalborg,...,KUN,2023-06-04,57.093056,9.850000,54.963919,24.084778,Aalborg,Kaunas,Monopoly always,913.71
4,205.95,Aalborg,AAL,Kaunas,KUN,2023-06-04T09:40:00,2023-06-04T17:55:00,scandinavian_airlines,435,Aalborg,...,KUN,2023-06-04,57.093056,9.850000,54.963919,24.084778,Aalborg,Kaunas,Monopoly always,913.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172388,299.75,Maastricht,MST,Barcelona,BCN,2023-06-08T14:55:00,2023-06-09T10:25:00,klm,1170,Maastricht/Aachen (NL),...,BCN,2023-06-08,50.911111,5.773611,41.297324,2.083316,Maastricht/Aachen (NL),Barcelona (ES),Monopoly always,1105.85
172389,339.90,Maastricht,MST,Barcelona,BCN,2023-06-08T14:55:00,2023-06-09T11:40:00,air_france,1245,Maastricht/Aachen (NL),...,BCN,2023-06-08,50.911111,5.773611,41.297324,2.083316,Maastricht/Aachen (NL),Barcelona (ES),Monopoly always,1105.85
172390,471.56,Maastricht,MST,Barcelona,BCN,2023-06-08T14:55:00,2023-06-09T09:00:00,air_france,1085,Maastricht/Aachen (NL),...,BCN,2023-06-08,50.911111,5.773611,41.297324,2.083316,Maastricht/Aachen (NL),Barcelona (ES),Monopoly always,1105.85
172391,19.61,Orebro,ORB,London Stansted,STN,2023-06-05T23:30:00,2023-06-06T00:40:00,ryanair,130,Orebro,...,LOND,2023-06-05,59.223733,15.037956,51.504117,-0.094347,Orebro,London (GB),Monopoly always,1279.74


In [287]:
# Choose the relevant columns from the data frame and melt it
relevant_cols = [
    "origin_city_search_term", "arrival_city_search_term", "route_competition_status",
    "origin_city_id", "latitude_origin_city", "longitude_origin_city", "origin_airport_name", "origin_airport_display_code",
    "arrival_city_id", "latitude_arrival_city", "longitude_arrival_city",  "arrival_airport_name", "arrival_airport_display_code",
    "competitor", "price_eur", "flight_duration", "flight_distance_km", "crawling_date"
]

# Sort the dataset by "crawling_date", "origin_city_search_term", "arrival_city_search_term", "competitor", "price_eur
sort_cols = ["crawling_date", "origin_city_search_term", "arrival_city_search_term", "competitor", "price_eur", "flight_duration"]
df_crawled_routes = df_crawled_routes[relevant_cols].sort_values(sort_cols).drop_duplicates().reset_index(drop=True)

# Pick the cheapest price_eur in each partition
# We don't include the airports, so we can get one price_eur per competitor
partition_1_cols = [
    "crawling_date", "origin_city_search_term", "origin_city_id", "latitude_origin_city", "longitude_origin_city",
    "arrival_city_search_term", "arrival_city_id", "latitude_arrival_city", "longitude_arrival_city",
    "route_competition_status", "competitor",
]
df_min_price_per_comp_route = df_crawled_routes.groupby(partition_1_cols, as_index=False)["price_eur"].min()

# Inner join df_min_price_per_comp_route and df_final to eliminate the more expensive flights
df_final = pd.merge(
    left=df_min_price_per_comp_route,
    right=df_crawled_routes[["origin_city_search_term", "arrival_city_search_term", "flight_distance_km"]].drop_duplicates(),
    on=["origin_city_search_term", "arrival_city_search_term"],
    how="inner"
)
df_final

,crawling_date,origin_city_search_term,origin_city_id,latitude_origin_city,longitude_origin_city,arrival_city_search_term,arrival_city_id,latitude_arrival_city,longitude_arrival_city,route_competition_status,competitor,price_eur,flight_distance_km
0,2023-06-04,Aalborg,AAL,57.093056,9.850000,Kaunas,KUN,54.963919,24.084778,Monopoly always,dat,139.79,913.71
1,2023-06-04,Aalborg,AAL,57.093056,9.850000,Kaunas,KUN,54.963919,24.084778,Monopoly always,klm,310.00,913.71
2,2023-06-04,Aalborg,AAL,57.093056,9.850000,Kaunas,KUN,54.963919,24.084778,Monopoly always,norwegian,113.94,913.71
3,2023-06-04,Aalborg,AAL,57.093056,9.850000,Kaunas,KUN,54.963919,24.084778,Monopoly always,ryanair,385.82,913.71
4,2023-06-04,Aalborg,AAL,57.093056,9.850000,Kaunas,KUN,54.963919,24.084778,Monopoly always,scandinavian_airlines,165.88,913.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39201,2023-06-05,Pardubice,PED,50.013333,15.738611,Alicante,ALC,38.285466,-0.560002,Monopoly always,ryanair,96.11,1833.97
39202,2023-06-08,Maastricht/Aachen (NL),MST,50.911111,5.773611,Barcelona (ES),BCN,41.297324,2.083316,Monopoly always,air_france,339.90,1105.85
39203,2023-06-08,Maastricht/Aachen (NL),MST,50.911111,5.773611,Barcelona (ES),BCN,41.297324,2.083316,Monopoly always,klm,299.75,1105.85
39204,2023-06-09,Barcelona (ES),BCN,41.297324,2.083316,Maastricht/Aachen (NL),MST,50.911111,5.773611,Monopoly always,air_france,239.05,1105.85


In [288]:
# Change the data frame so that each competitor's price is shown as one column
pivot_table_index_cols = [
    "crawling_date", "origin_city_search_term", "origin_city_id", "latitude_origin_city", "longitude_origin_city",
    "arrival_city_search_term", "arrival_city_id", "latitude_arrival_city", "longitude_arrival_city", "route_competition_status", "flight_distance_km"
] 
df_final_pivot = pd.pivot_table(
    data=df_final,
    values="price_eur",
    index=pivot_table_index_cols,
    columns=["competitor"]
).reset_index()

# Display Ryan Air and Wizz as the first two competitors
all_df_final_pivot_cols = df_final_pivot.columns.tolist()
first_cols = pivot_table_index_cols + ["ryanair", "wizz"]
exclusion_list = list(set(all_df_final_pivot_cols) - set(first_cols))
rearranged_col_list = first_cols + exclusion_list 
df_final_pivot = df_final_pivot[rearranged_col_list]
# Remove the name "competitor" from the index
df_final_pivot.rename_axis(None, inplace=True, axis=1)

# Display the final table
df_final_pivot

,crawling_date,origin_city_search_term,origin_city_id,latitude_origin_city,longitude_origin_city,arrival_city_search_term,arrival_city_id,latitude_arrival_city,longitude_arrival_city,route_competition_status,...,transavia,czech_airlines,aer_lingus,icelandair,air_india,bulgarian_air_charter,blue_islands,china_eastern,ethiopian_airlines,citizenplane
0,2023-06-04,Aalborg,AAL,57.093056,9.850000,Kaunas,KUN,54.963919,24.084778,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-06-04,Aalborg,AAL,57.093056,9.850000,London (GB),LOND,51.504117,-0.094347,Monopoly not always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-06-04,Aalborg,AAL,57.093056,9.850000,Stockholm,STOC,59.329474,18.062640,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-06-04,Aarhus,AAR,56.310278,10.618056,Faro (PT),FAO,46.915556,-96.814722,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-06-04,Aarhus,AAR,56.310278,10.618056,Gdansk,GDN,54.376558,18.470078,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5107,2023-06-05,London (GB),LOND,51.504117,-0.094347,Orebro,ORB,59.223733,15.037956,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5108,2023-06-05,Orebro,ORB,59.223733,15.037956,London (GB),LOND,51.504117,-0.094347,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5109,2023-06-05,Pardubice,PED,50.013333,15.738611,Alicante,ALC,38.285466,-0.560002,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5110,2023-06-08,Maastricht/Aachen (NL),MST,50.911111,5.773611,Barcelona (ES),BCN,41.297324,2.083316,Monopoly always,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [289]:
# Upload the data to BigQuery
credentials = service_account.Credentials.from_service_account_file(
    filename=os.path.expanduser("~") + "/bq_credentials.json", scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
client = bigquery.Client(project="web-scraping-371310", credentials=credentials)
job_config_raw = bigquery.LoadJobConfig()
job_config_raw.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

job_config_pivot = bigquery.LoadJobConfig()
job_config_pivot.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Upload the raw data
client.load_table_from_dataframe(
    dataframe=df_crawled_routes,
    destination="web-scraping-371310.crawled_datasets.benoit_flight_route_data_raw",
    job_config=job_config_pivot
).result()

# Upload the pivoted table
client.load_table_from_dataframe(
    dataframe=df_final_pivot,
    destination="web-scraping-371310.crawled_datasets.benoit_flight_route_data_pivoted",
    job_config=job_config_pivot
).result()

LoadJob<project=web-scraping-371310, location=EU, id=9fe72772-e2f8-47bc-9a76-0ec4032da64c>